In [1]:
!pip install pdfplumber pandas tqdm python-docx regex transformers datasets peft accelerate safetensors sentencepiece
# If you need OCR (scanned pages), also:
!pip install pytesseract pdf2image pillow
# Note: on Kaggle, you may need apt packages for tesseract: apt-get install -y tesseract-ocr (not available in all runtimes)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s e

In [2]:
import re, json, os, unicodedata
from datetime import datetime
from tqdm import tqdm
import pdfplumber
import pandas as pd
from docx import Document

# Optional transformer summarizer
try:
    from transformers import pipeline
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    HAS_SUMMARY = True
except Exception as e:
    print("Transformer summarizer not available:", e)
    summarizer = None
    HAS_SUMMARY = False

2025-11-04 14:09:26.144850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762265366.563034      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762265366.684542      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [14]:
# ---------- CONFIG ----------
PDF_PATH = "/kaggle/input/modifycl/modifypdf.pdf"
DOCX_PATH = "/kaggle/input/just-to-make-it-work/AI for Mine Safety Intelligence (accident pattern detection).docx"
OUT_DIR = "/kaggle/working"
OUT_CSV = os.path.join(OUT_DIR, "accidents_phase1.csv")
OUT_JSONL = os.path.join(OUT_DIR, "accidents_phase1.jsonl")
OUT_META = os.path.join(OUT_DIR, "accidents_phase1_metadata.json")

USE_TRANSFORMER_SUMMARY = True
SUMMARY_MAX_TOKENS = 150
# ----------------------------

In [4]:
# Regex tuned to DGMS-like headers (adjust if your PDF differs)
def clean_text(t):
    """normalize unicode, strip headers/footers, fix hyphens"""
    if not t:
        return ""
    t = unicodedata.normalize("NFKC", t)
    t = re.sub(r'(\w+)-\n(\w+)', r'\1\2', t)      # fix word breaks
    t = re.sub(r'(?<!\n)\n(?!\n)', ' ', t)        # join single newlines
    t = re.sub(r'\s{2,}', ' ', t)
    return t.strip()

In [5]:

def norm_date(d):
    d = (d or "").strip().replace('.', '/').replace('-', '/')
    for fmt in ("%d/%m/%y", "%d/%m/%Y", "%Y/%m/%d"):
        try:
            return datetime.strptime(d, fmt).strftime("%Y-%m-%d")
        except: pass
    return d

In [6]:
def norm_time(t):
    t = (t or "").strip().replace('.', ':')
    if not t: return ""
    try:
        if ':' in t:
            return datetime.strptime(t, "%H:%M").strftime("%H:%M")
        else:
            return datetime.strptime(t, "%H").strftime("%H:%M")
    except: return t

In [7]:
def simple_summary(text, max_chars=250):
    sents = re.split(r'(?<=[.!?])\s+', text.strip())
    return " ".join(sents[:2])[:max_chars] if sents else text[:max_chars]

In [8]:
RE_HEADER = re.compile(
    r"Date\s*[-:]\s*(?P<date>[\d/]{6,10}).*?Mine\s*[-:]\s*(?P<mine>.*?)"
    r"(?:Time\s*[-:]\s*(?P<time>[\d:\.]{1,5}))?.*?"
    r"Owner\s*[-:]\s*(?P<owner>.*?)"
    r"(?:Dist\.\s*[-:]\s*(?P<dist>.*?),\s*State\s*[-:]\s*(?P<state>.*?))?"
    r"(?:Person\(s\)\s*Killed\s*[:\-]\s*(?P<persons>.*?))?",
    re.IGNORECASE | re.DOTALL)

In [9]:

def extract_from_pdf(pdf_path):
    """Parse pages and extract structured accident blocks"""
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for pageno, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            if not text.strip():
                continue
            text = clean_text(text)
            # split by 'Date - ' lookahead
            blocks = re.split(r'(?=Date\s*[-:]\s*\d{1,2}[/\-]\d{1,2}[/\-]\d{2,4})', text, flags=re.I)
            for block in blocks:
                if len(block.strip()) < 60: continue
                m = RE_HEADER.search(block)
                if not m: continue
                gd = m.groupdict()
                date = norm_date(gd.get("date"))
                time = norm_time(gd.get("time"))
                row = {
                    "source": os.path.basename(pdf_path),
                    "page": pageno,
                    "date": date,
                    "time": time,
                    "mine": (gd.get("mine") or "").strip(),
                    "owner": (gd.get("owner") or "").strip(),
                    "district": (gd.get("dist") or "").strip(),
                    "state": (gd.get("state") or "").strip(),
                    "persons_raw": (gd.get("persons") or "").strip(),
                    "narrative": block[m.end():].strip()
                }
                rows.append(row)
    return rows

In [10]:
def read_docx_description(docx_path):
    if not os.path.exists(docx_path): return ""
    try:
        doc = Document(docx_path)
        return "\n".join(p.text.strip() for p in doc.paragraphs if p.text.strip())
    except Exception as e:
        print("Docx read failed:", e)
        return ""

In [11]:
def make_prompt_completion(row):
    prompt = ("Summarize and extract key fields (date, time, mine, owner, district, state, fatalities, summary) "
              "from the following mining accident report:\n\n" + row["narrative"])
    completion = json.dumps({
        "date": row["date"],
        "time": row["time"],
        "mine": row["mine"],
        "owner": row["owner"],
        "district": row["district"],
        "state": row["state"],
        "fatalities": row["fatalities"],
        "summary": row["summary"]
    }, ensure_ascii=False)
    return prompt, completion


In [15]:
def main():
    print("📄 Extracting records from:", PDF_PATH)
    pdf_rows = extract_from_pdf(PDF_PATH)
    print(f"Found {len(pdf_rows)} raw candidate records.")

    processed = []
    for i, r in enumerate(tqdm(pdf_rows)):
        narrative = (r.get("narrative") or "").strip()
        persons_raw = r.get("persons_raw", "")
        # fatalities
        deaths = 0
        mdeath = re.search(r'\b(\d+)\s*Death', narrative, flags=re.I)
        if mdeath: deaths = int(mdeath.group(1))
        elif persons_raw: deaths = len(re.findall(r'\d+\.', persons_raw)) or 1

        # summary
        if HAS_SUMMARY and narrative:
            try:
                s = summarizer(narrative, max_length=SUMMARY_MAX_TOKENS, min_length=25, truncation=True)
                summary = s[0]["summary_text"].strip()
            except Exception:
                summary = simple_summary(narrative)
        else:
            summary = simple_summary(narrative)

        processed.append({
            "id": f"rec_{i+1}",
            "source": r["source"],
            "page": r["page"],
            "date": r["date"],
            "time": r["time"],
            "mine": r["mine"],
            "owner": r["owner"],
            "district": r["district"],
            "state": r["state"],
            "fatalities": deaths,
            "persons_raw": persons_raw,
            "narrative": narrative,
            "summary": summary
        })

    # ---------- SAVE ----------
    df = pd.DataFrame(processed)
    os.makedirs(OUT_DIR, exist_ok=True)
    df.to_csv(OUT_CSV, index=False)
    print(f"✅ CSV saved -> {OUT_CSV} ({len(df)} rows)")

    with open(OUT_JSONL, "w", encoding="utf-8") as f:
        for r in processed:
            prompt, completion = make_prompt_completion(r)
            f.write(json.dumps({"prompt": prompt, "completion": completion}, ensure_ascii=False) + "\n")
    print(f"✅ JSONL saved -> {OUT_JSONL}")

    description = read_docx_description(DOCX_PATH)
    meta = {
        "source_pdf": os.path.basename(PDF_PATH),
        "description_excerpt": description[:500],
        "num_records": len(processed),
        "created_at": datetime.utcnow().isoformat() + "Z"
    }
    with open(OUT_META, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2, ensure_ascii=False)
    print(f"✅ Metadata saved -> {OUT_META}")

    print("\nSample record preview:\n", json.dumps(processed[0], indent=2, ensure_ascii=False)[:800])



In [16]:
if __name__ == "__main__":
    main()

📄 Extracting records from: /kaggle/input/modifycl/modifypdf.pdf
Found 45 raw candidate records.


100%|██████████| 45/45 [00:41<00:00,  1.08it/s]

✅ CSV saved -> /kaggle/working/accidents_phase1.csv (45 rows)
✅ JSONL saved -> /kaggle/working/accidents_phase1.jsonl
✅ Metadata saved -> /kaggle/working/accidents_phase1_metadata.json

Sample record preview:
 {
  "id": "rec_1",
  "source": "modifypdf.pdf",
  "page": 1,
  "date": "2015-05-16",
  "time": "",
  "mine": "",
  "owner": "",
  "district": "",
  "state": "",
  "fatalities": 0,
  "persons_raw": "",
  "narrative": "HINDUSTAN COPPER LTD. Dist. - Jhunjhunu, State - Rajasthan Person(s) Killed : 1. Vijendra Singh, Driller, Male, 32 Years While two drillers were connecting compressed air hose-pipe to jack hammer drill machine at a distance of 3.5m from the development face in an underground Metalliferous Mine, a mass of stone measuring about 1.0m(length) x 0.5m(width) x 0.35m (thick)fell from the unsupported roof from a height of about 4.0m on one of the drillers inflicting serious bodily injuries to him to which he succumbed after about seven hours. Had the workings within 9.0m of 